# Task 4

In [ ]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import pipeline
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# --- Class: Retriever ---
class ComplaintRetriever:
    def __init__(self, vector_store_path, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={"device": device},
            encode_kwargs={"normalize_embeddings": True}
        )
        self.db = FAISS.load_local(
            vector_store_path,
            self.embeddings,
            allow_dangerous_deserialization=True
        )

    def retrieve(self, query, top_k=5):
        return self.db.similarity_search(query, k=top_k)

# --- Class: Generator ---
class ComplaintAnswerGenerator:
    def __init__(self, model_name="google/flan-t5-base"):
        device = 0 if torch.cuda.is_available() else -1
        self.pipe = pipeline("text2text-generation", model=model_name, device=device)

    def format_prompt(self, context_chunks, question):
        context_text = "\n\n".join(context_chunks)
        return (
            "You are a financial analyst assistant for CrediTrust. "
            "Use the following retrieved complaint excerpts to answer the question. "
            "If the context doesn't contain the answer, say you don't have enough information.\n\n"
            f"Context:\n{context_text}\n\n"
            f"Question: {question}\nAnswer:"
        )

    def generate_answer(self, context_chunks, question):
        prompt = self.format_prompt(context_chunks, question)
        output = self.pipe(prompt, max_new_tokens=256)[0]["generated_text"]
        return output.strip()

# --- Summarizer ---
@st.cache_resource
def get_summarizer():
    return pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def summarize_answer(text):
    summarizer = get_summarizer()
    summary = summarizer(text, max_length=120, min_length=40, do_sample=False)
    return summary[0]['summary_text']

# --- RAG Pipeline ---
class RAGPipeline:
    def __init__(self, vector_store_path):
        self.retriever = ComplaintRetriever(vector_store_path)
        self.generator = ComplaintAnswerGenerator()

    def ask(self, question):
        retrieved_chunks = self.retriever.retrieve(question, top_k=5)
        context_chunks = [doc.page_content for doc in retrieved_chunks]
        answer = self.generator.generate_answer(context_chunks, question)
        return answer, context_chunks

# --- Streamlit App ---
st.set_page_config(page_title="CrediTrust Complaint Chatbot", layout="centered")
st.title("🤖 CrediTrust Complaint Chatbot")
st.caption("Retrieve and answer customer complaint queries.")

VECTOR_STORE_PATH = "/content/drive/MyDrive/vector_store/faiss_index_sample"

if "rag" not in st.session_state:
    with st.spinner("🔄 Loading model and vector store..."):
        st.session_state.rag = RAGPipeline(VECTOR_STORE_PATH)

question = st.text_input("💬 Ask a question about customer complaints:")

if st.button("📤 Submit") and question.strip():
    with st.spinner("🔍 Retrieving and answering..."):
        answer, sources = st.session_state.rag.ask(question)
        summarized = summarize_answer(answer)

    st.success("✅ Summarized Answer")
    st.markdown(f"**{summarized}**")

    with st.expander("📚 Show Retrieved Sources"):
        for i, src in enumerate(sources, 1):
            st.markdown(f"**Source {i}:** {src[:500]}{'...' if len(src) > 500 else ''}")

if st.button("♻️ Clear"):
    st.experimental_rerun()


Overwriting app.py


In [ ]:
!pip install streamlit transformers langchain langchain-huggingface langchain-community faiss-cpu


In [ ]:
!nohup streamlit run app.py --server.port 8501 --server.headless true &
from pyngrok import ngrok
ngrok.set_auth_token("2xZlUxfhFUbZzDsSJ2KAvpeFMB2_2xFk4C4wUXP8mVkPtH3ci")
print(f"Streamlit public URL: {public_url}")


nohup: appending output to 'nohup.out'
Streamlit public URL: NgrokTunnel: "https://edf778d068e1.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!git clone https://github.com/yesufma/credi-trust-rag-chatbot.git
%cd credi-trust-rag-chatbot
